In [1]:
from __future__ import print_function
import torch
import torchvision as tv

from torchvision import datasets, models
from torchvision import transforms
from torch.autograd import Variable
from torch.nn.utils.rnn import pack_padded_sequence
import torch.optim as optim
import torch.nn as nn

from data_loader import get_coco_data_loader
import os


from vocab import Vocabulary, load_vocab
from decoder_LSTM import decoderLSTM
import utils
from models import CNN

import nltk
#nltk.download('punkt')

cuda


In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu' 
#print(device)

In [3]:
# define hyper-parameters
batch_size = 128 
num_workers = 1
learning_rate = 1e-3
epoches = 3


embedding_size = 512 # same as paper
hidden_size = 512 
vocab_size = 15000  ### to be filled in
beam_size = 20
dropout = 0.5
evaluate_step = 120  # check evaluate each 128 steps 

GPU_num = 1 # how many GPU in use
save_step = 500

In [4]:
# define path
dataset_image_root_dir = '/datasets/COCO-2015/'
dataset_image_train = 'train2014'
dataset_image_val = 'val2014'
dataset_image_test = 'test2014'

dataset_annotation_root_dir = '/datasets/ee285f-public/COCO-Annotations/'
dataset_annotation_train = 'annotations_trainval2014/captions_train2014.json'
dataset_annotation_val = 'annotations_trainval2014/captions_val2014.json'
dataset_annotation_test = 'image_info_test2015/image_info_test2015.json'

In [5]:
# image processing
# Image preprocessing, normalization for the pretrained resnet
transform = tv.transforms.Compose([
        tv.transforms.Resize((224,224)),# resize the input Image to the given size.
        tv.transforms.ToTensor(), # convert to tensor, from [0,255]->[0,1]
        tv.transforms.Normalize(mean=[0.5,0.5,0.5],std=[0.5,0.5,0.5]) #input[channel] = (input[channel] - mean[channel]) / std[channel],3 channels 
])


In [6]:
# load COCOs dataset
vocab = load_vocab()

train_loader = get_coco_data_loader(path= dataset_image_root_dir+dataset_image_train,
                                    json= dataset_annotation_root_dir + dataset_annotation_train ,
                                    vocab=vocab,
                                    transform=transform,
                                    batch_size=batch_size,
                                    shuffle=True,
                                    num_workers=num_workers)


val_loader = get_coco_data_loader(path= dataset_image_root_dir+dataset_image_val,
                                  json= dataset_annotation_root_dir+dataset_annotation_val,
                                  vocab=vocab,
                                  transform=transform,
                                  batch_size=batch_size,
                                  shuffle=False,
                                  num_workers=num_workers)

loading annotations into memory...
Done (t=0.63s)
creating index...
index created!
loading annotations into memory...
Done (t=0.33s)
creating index...
index created!


In [7]:
losses_train = []
losses_val = []

encoder = CNN(embedding_size).to(device)  # TO be complete
decoder = decoderLSTM(embedding_size,hidden_size,vocab_size,beam_size = 20,dropout=0.5).to(device)

# Loss
criterion = nn.CrossEntropyLoss()

In [8]:
checkpoint_file = None
checkpoint_dir = 'checkpoints'
sample = False

if checkpoint_file:
    encoder_state_dict, decoder_state_dict, optimizer, *meta = utils.load_models(checkpoint_file,sample)
    initial_step, initial_epoch, losses_train, losses_val = meta
    encoder.load_state_dict(encoder_state_dict)
    decoder.load_state_dict(decoder_state_dict)
else:
    params = list(decoder.parameters()) + list(encoder.linear.parameters()) + list(encoder.batchnorm.parameters())
    optimizer = torch.optim.Adam(params, lr=learning_rate)


if sample == True:
    utils.sample(encoder, decoder, vocab, val_loader)

In [9]:
# Train the model
total_step = len(train_loader)  # each step corresponds to a minibatch - 3236
try:
    for epochInd in range(epoches):
        for step, (images, captions, lengths) in enumerate(train_loader):

            # the format depends on data_loader
            images = images.to(device)
            captions = captions.to(device)
            targets = pack_padded_sequence(captions, lengths, batch_first=True)[0]

            # forward 
            if GPU_num > 1:
                # run on multiple GPU
                features = nn.parallel.data_parallel(encoder, images, range(GPU_num))
                outputs = nn.parallel.data_parallel(decoder, features, range(GPU_num))
            else:
                # run on single GPU
                features = encoder(images)
                outputs = decoder(features, captions, lengths)
            #print("one_step")

            # Initialize zero gradient
            decoder.zero_grad()
            encoder.zero_grad()

            # loss
            loss = criterion(outputs, targets) # tensor(9.6140, device='cuda:0', grad_fn=<NllLossBackward>)
            losses_train.append(loss.item())  # save losses

            # backpropagate
            loss.backward()

            # update the parameter
            optimizer.step()

            # Debugging - show trend
            print("Training: {}".format(loss.item()))


            '''
            # check the performance on validation dataset
            if step % evaluate_step == 0 and step!=0:
                # change to val mode
                encoder.eval()
                decoder.eval()

                batch_losses_val = []
                with torch.no_grad():

                    for val_step, (images, captions, lengths) in enumerate(val_loader):
                        images = images.to(device)
                        captions = captions.to(device)
                        targets = pack_padded_sequence(captions, lengths, batch_first=True)[0]
                        # forward 
                        if GPU_num > 1:
                            # run on multiple GPU
                            features = nn.parallel.data_parallel(encoder, images, range(GPU_num))
                            outputs = nn.parallel.data_parallel(decoder, features, range(GPU_num))
                        else:
                            # run on single GPU
                            features = encoder(images)
                            outputs = decoder(features, captions, lengths)
                        batch_loss_val = criterion(outputs, targets) 

                        #print("Cur Valid ")
                        #print(batch_loss_val)
                        batch_losses_val.append(batch_loss_val.item()) 

                    # compute the mean value across the whole batch
                    mean_losses_val = np.mean(batch_losses_val)
                    losses_val.append(mean_losses_val)


                    # predict during the validation process for the last minibatch
                    sampled_ids = decoder.sample(features)
                    sampled_ids = sampled_ids.cpu().data.numpy()[0] # only show result for 1st image
                    # need further check !!!
                    sentence = utils.convert_back_to_text(sampled_ids, vocab)
                    print('Sample Results:', sentence)

                    true_ids = captions.cpu().data.numpy()[0]
                    sentence = utils.convert_back_to_text(true_ids, vocab)
                    print('Targeted Results:', sentence)

                ############## You can print the losses #####################     
                # change to train mode
                encoder.train()
                decoder.train() '''
                
            # Save the model
            if (step+1) % save_step == 0:
                utils.save_models(encoder, decoder, optimizer, step, epoches, losses_train, losses_val, checkpoint_dir)
                utils.dump_losses(losses_train, losses_val, os.path.join(checkpoint_dir, 'losses.pkl'))

                
except KeyboardInterrupt:
    pass

finally:
    # Do final save
    utils.save_models(encoder, decoder, optimizer, step, epoches, losses_train, losses_val, checkpoint_dir)
    utils.dump_losses(losses_train, losses_val, os.path.join(checkpoint_dir, 'losses.pkl'))             

Training: 9.625948905944824
Training: 9.40645694732666
Training: 9.166086196899414
Training: 8.852590560913086
Training: 8.437247276306152
Training: 7.826584815979004
Training: 6.997297286987305
Training: 6.147791862487793
Training: 5.622804164886475
Training: 5.243687152862549
Training: 5.029626846313477
Training: 4.973631858825684
Training: 4.806797981262207
Training: 4.745721340179443
Training: 4.899274826049805
Training: 4.854737281799316
Training: 4.79506254196167
Training: 4.7793192863464355
Training: 4.8086018562316895
Training: 4.79522705078125
Training: 4.603905200958252
Training: 4.658043384552002
Training: 4.751053810119629
Training: 4.465237140655518
Training: 4.5507097244262695
Training: 4.694652080535889
Training: 4.495165824890137
Training: 4.534500598907471
Training: 4.4490132331848145
Training: 4.526712894439697
Training: 4.485520362854004
Training: 4.471712589263916
Training: 4.416699409484863
Training: 4.418001651763916
Training: 4.341322898864746
Training: 4.3311514

In [10]:
# change to val mode & check temp results
encoder.eval()
decoder.eval()

batch_losses_val = []
with torch.no_grad():

    for val_step, (images, captions, lengths) in enumerate(val_loader):
        images = images.to(device)
        captions = captions.to(device)
        targets = pack_padded_sequence(captions, lengths, batch_first=True)[0]
        # forward 
        if GPU_num > 1:
            # run on multiple GPU
            features = nn.parallel.data_parallel(encoder, images, range(GPU_num))
            outputs = nn.parallel.data_parallel(decoder, features, range(GPU_num))
        else:
            # run on single GPU
            features = encoder(images)
            outputs = decoder(features, captions, lengths)
        batch_loss_val = criterion(outputs, targets) 

        #print("Cur Valid ")
        #print(batch_loss_val)
        batch_losses_val.append(batch_loss_val.item()) 

        

        # predict during the validation process for the last minibatch
        sampled_ids = decoder.sample(features)
        sampled_ids = sampled_ids.cpu().data.numpy()[0] # only show result for 1st image
        # need further check !!!
        sentence = utils.convert_back_to_text(sampled_ids, vocab)
        print('Sample Results:', sentence)

        true_ids = captions.cpu().data.numpy()[0]
        sentence = utils.convert_back_to_text(true_ids, vocab)
        print('Targeted Results:', sentence)
        
        
    # compute the mean value across the whole batch
    mean_losses_val = np.mean(batch_losses_val)
    losses_val.append(mean_losses_val)

Sample Results: a man with a table .
Targeted Results: this is a blue and white bathroom with a wall sink and a <unk> on the wall .
Sample Results: a man with a table .
Targeted Results: a person on a bench , and one on a wheelchair sitting by a <unk> looking out toward the ocean
Sample Results: a man with a table .
Targeted Results: a blue door with a painting of a cat 's rear end entering thru a cat door .


KeyboardInterrupt: 